# Data generation script

In [3]:
import matplotlib.pylab as plt
import numpy as np
import scipy
import soundfile as sf
from IPython.display import Audio 
import os
import tqdm
from os.path import join as pjoin
import scipy.signal as sig

In [4]:
def stereo_conv(audio, rir):
    #shape is CHANNELS, SAMPLE
    l = scipy.signal.fftconvolve(audio[0,:], rir[0,:], 'valid')
    r = scipy.signal.fftconvolve(audio[1,:], rir[1,:], 'valid')        
    return np.array([l, r])

def power(signal):
    return np.mean(signal**2)

def plot_stereo(signal):
    plt.plot(signal[0,:])
    plt.plot(signal[1,:])



def synch_sigs(sig1,sig2):
    sig1_out=np.zeros(sig1.shape)
    sig2_out=np.zeros(sig2.shape)
    corr = scipy.signal.correlate(sig1, sig2, 'full')
    lag = scipy.signal.correlation_lags(len(sig1), len(sig2), mode='full')[np.argmax(corr)]
    if lag > 0:
        sig2=sig2[0:-lag]
        sig1=sig1[lag:]
    elif lag < 0:
        sig2=sig2[-lag:]
        sig1=sig1[0:lag]

    sig1_out[:sig1.shape[0]]=sig1
    sig2_out[:sig2.shape[0]]=sig2
    return sig1_out,sig2_out, lag


def gen_out(room):
    # if Beyer MM1 recording is available use it, otherwhise take the first channel from the Zylia mic
    if os.path.exists(os.path.join(os.path.join(DATA_PATH, room), 'beyer.wav')):
        path = os.path.join(os.path.join(DATA_PATH, room), 'beyer.wav')
        iszylia = False
    else:
        path = os.path.join(os.path.join(DATA_PATH, room), 'zylia.wav')
        iszylia = True
    wet_sweep, fs2 = sf.read(path)
    if len(wet_sweep.shape) > 1:
        wet_sweep = wet_sweep[:, 0]

    # resample if needed
    if fs2 != fs:
        print('Resampling '+room)
        gcd = np.gcd(fs2, fs)
        wet_sweep = scipy.signal.resample_poly(wet_sweep, fs // gcd, fs2 // gcd)

    # crop Left and Right PA RIRs
    wet_sweep_L = wet_sweep[:T*fs]
    wet_sweep_R = wet_sweep[2*T*fs:3*T*fs]

    # compute the inverse filter and obtain the rir
    s_inv = np.flip(SWEEP, 0) / np.exp(AXIS/L)
    h_l = scipy.signal.fftconvolve(wet_sweep_L, s_inv)
    h_r = scipy.signal.fftconvolve(wet_sweep_R, s_inv)

    #sync left and right rirs (virtually center the mic in the room)
    nh_l, nh_r, lag = synch_sigs(h_l, h_r)

    
    #use a smoothed envelope for cropping before and after the RIR falls into the noise floor
    rir = np.vstack((nh_l, nh_r))    
    envelope = np.abs(sig.hilbert(rir[1]))
    smoothing = 100
    envelope = sig.fftconvolve(envelope, 1/smoothing * np.ones(smoothing), 'same')
    env_max = np.argmax(envelope)
    noise_floor = envelope[env_max - int(2*fs*0.1) : env_max- int(fs*0.1)]
    noise_floor_avg = np.mean(noise_floor)
    
    # we crop from 100ms before direct sound
    start_idx = env_max - int(fs*0.1)
    
    # apply a fade in for the first 50ms
    rir = rir[:, start_idx:]
    fadein = np.hstack((np.linspace(0, 1, int(env_max - start_idx) // 2), np.ones(rir.shape[1] - int(env_max - start_idx) // 2)))
    rir[0] *= fadein
    rir[1] *= fadein

    # crop after if falls into the noise floor
    envelope = envelope[start_idx:]
    end_idx = np.where(envelope[int(fs*0.1):] < noise_floor_avg)[0][0]
    rir = rir[:, :end_idx]
    thlds = np.where(envelope < noise_floor_avg)[0]
    end_idx = thlds[thlds>int(fs*0.1)][0]
    envelope = envelope[:end_idx]
    rir = rir[:, :end_idx]

    # apply a fade out to avoid clicks
    fadeout = np.hstack((np.ones(rir.shape[1] - int(env_max - start_idx) // 2), np.linspace(1, 0, int(env_max - start_idx) // 2)))
    rir[0] *= fadeout
    rir[1] *= fadeout

    if iszylia:
        #print('EQing RIR. '+room)
        rir = sig.sosfilt(SOS, rir)
    
    #normalize
    global_max = np.max((np.abs(rir[0]), np.abs(rir[1])))
    rir[0] /= np.max(np.abs(rir[0])) / global_max
    rir[1] /= np.max(np.abs(rir[1])) / global_max
    rir /= np.max(np.abs(rir))

    # apply to the test track for generating a sample
    #out = stereo_conv(TEST_TRACK, rir)
    
    #out *= np.sqrt(power(test_track) / power(out)) 
    
    split_idx = np.argmax(rir[0])+int(0.002 * fs)
    
    #return out, rir
    return rir, split_idx

def design_high_shelf(fs, f0, gain_db, Q=0.707):
    """
    Designs a high-shelf filter using RBJ audio EQ cookbook formulas.
    Returns a second-order section (sos) array.
    
    fs      : Sampling frequency [Hz]
    f0      : Shelf corner frequency [Hz]
    gain_db : Gain at high frequencies [dB] (negative = cut)
    Q       : Quality factor (controls transition slope)
    """
    A = 10**(gain_db/40)  # convert dB gain to linear
    w0 = 2*np.pi*f0/fs
    alpha = np.sin(w0)/(2*Q)
    cosw0 = np.cos(w0)

    # RBJ cookbook high-shelf coefficients
    b0 =    A*((A+1) + (A-1)*cosw0 + 2*np.sqrt(A)*alpha)
    b1 = -2*A*((A-1) + (A+1)*cosw0)
    b2 =    A*((A+1) + (A-1)*cosw0 - 2*np.sqrt(A)*alpha)
    a0 =       (A+1) - (A-1)*cosw0 + 2*np.sqrt(A)*alpha
    a1 =  2*((A-1) - (A+1)*cosw0)
    a2 =       (A+1) - (A-1)*cosw0 - 2*np.sqrt(A)*alpha

    # Normalize coefficients
    b = np.array([b0, b1, b2]) / a0
    a = np.array([1, a1/a0, a2/a0])

    # Convert to SOS for stability
    sos = sig.tf2sos(b, a)
    return sos

In [ ]:
MPATH = '/media/diskA/enric/parirset'    
osweep, fs = sf.read('sweep.wav')
T = 6          # time duration of the sweep
AXIS = np.linspace(0,T,fs*T)  # time axis
L = T/np.log(20000/20)                   

test_track, fs3 = sf.read('test_track.wav')
TEST_TRACK = test_track.T

SWEEP = osweep[:T*fs, 0]
DATA_PATH = pjoin(MPATH, 'original_recordings')
RIRS_PATH = pjoin(MPATH, 'rirs')
SAMPLES_PATH = pjoin(MPATH, 'samples')

FS = 48000

SOS = design_high_shelf(fs, 3500, -4, Q=0.5)

In [ ]:
if not os.path.exists(SAMPLES_PATH):
    os.makedirs(SAMPLES_PATH)
if not os.path.exists(RIRS_PATH):
    os.makedirs(RIRS_PATH)   
rooms = os.listdir('original_recordings')

rooms.sort()

rooms = rooms[1:-5]

In [ ]:
for room in rooms:
    try:
        out, rir = gen_out(room)
        sf.write(pjoin(SAMPLES_PATH, room+'_sample.wav'), out.T, samplerate=fs)
        sf.write(pjoin(RIRS_PATH, room+'_rir.wav'), rir.T, samplerate=fs)
    except:
        print('Error in '+room)

In [ ]:
# finally the K2 RIR:
rir_L, fs2 = sf.read('original_recordings/40_K2/P1_EQ.wav')
rir_R, fs2 = sf.read('original_recordings/40_K2/P2_EQ.wav')

rir = np.vstack((rir_L, rir_R))

rir = rir[:, np.min((np.argmax(rir[0]), np.argmax(rir[1]))) - int(0.1*fs):]

global_max = np.max((np.abs(rir[0]), np.abs(rir[1])))

rir[0] /= np.max(np.abs(rir[0])) / global_max
rir[1] /= np.max(np.abs(rir[1])) / global_max
rir /= np.max(np.abs(rir))
out = stereo_conv(TEST_TRACK, rir)
out *= np.sqrt(power(test_track) / power(out)) 


sf.write(pjoin(SAMPLES_PATH, '40_K2.wav'), out.T, samplerate=fs)
sf.write(pjoin(RIRS_PATH, '40_K2.wav'), rir.T, samplerate=fs)

In [ ]:
# finally the K2 RIR:
rir_L, fs2 = sf.read('original_recordings/41_K2p3/P3_EQ.wav')
rir_R, fs2 = sf.read('original_recordings/41_K2p3/P4_EQ.wav')

rir = np.vstack((rir_L, rir_R))

rir = rir[:, np.min((np.argmax(rir[0]), np.argmax(rir[1]))) - int(0.1*fs):]

global_max = np.max((np.abs(rir[0]), np.abs(rir[1])))

rir[0] /= np.max(np.abs(rir[0])) / global_max
rir[1] /= np.max(np.abs(rir[1])) / global_max
rir /= np.max(np.abs(rir))
out = stereo_conv(TEST_TRACK, rir)
out *= np.sqrt(power(test_track) / power(out)) 


sf.write(pjoin(SAMPLES_PATH, '41_K2p3.wav'), out.T, samplerate=fs)
sf.write(pjoin(RIRS_PATH, '41_K2p3.wav'), rir.T, samplerate=fs)